# Quantifying Crowded Trades(QCT): A Comprehensive Analysis
### sector_relative_value
### Collins Emezie Ugwuozor, Jayesh Chandra Gupta, Emmanuel Effiong Asuquo

In [1]:
#Import the neccessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
from pandas_datareader import data
import yfinance as yf
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
%matplotlib inline
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import yahoo_fin
from yahoo_fin import stock_info

In [3]:
# Pick up table of S&P 500 listed companies from Wikipedia.
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [4]:
table = data[0]
table.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [5]:
table['GICS Sector'].unique()

array(['Industrials', 'Health Care', 'Information Technology',
       'Consumer Staples', 'Utilities', 'Financials',
       'Consumer Discretionary', 'Materials', 'Real Estate',
       'Communication Services', 'Energy'], dtype=object)

In [6]:
# Select sectors of S&P 500
industrials_sector = table[table['GICS Sector'] == 'Industrials'] # XLI
health_sector = table[table['GICS Sector'] == 'Health Care'] # XLV
tech_sector = table[table['GICS Sector'] == 'Information Technology'] # XLK
consumer_staples_sector = table[table['GICS Sector'] == 'Consumer Staples'] # XLP
utilities_sector = table[table['GICS Sector'] == 'Utilities'] # XLU
financial_sector = table[table['GICS Sector'] == 'Financials'] # XLF
consumer_discretionary_sector = table[table['GICS Sector'] == 'Consumer Discretionary'] # XLY
materials_sector = table[table['GICS Sector'] == 'Materials'] # XLB
real_estate_sector = table[table['GICS Sector'] == 'Real Estate'] # XLRE
communication_services_sector = table[table['GICS Sector'] == 'Communication Services'] # XLC *XTL
energy_sector = table[table['GICS Sector'] == 'Energy'] # XLE

In [7]:
# Define dates for five year historical prices
start_date = "2018-01-01"
end_date = "2023-01-01"

In [8]:
# Sector ETFs for centrality across sectors
symbols = ['XLB', 'XLI', 'XLY', 'XLP', 'XLE', 'XLV', 'XLF', 'XLK', 'XTL', 'XLU', 'XLRE']

In [9]:
# Sector's Mkt Cap (USD m) picked manually From bloomberg
market_cap = [5150,37211,29320,14101,52475,15373,4368,13622,35850,16512,48]

### Relative Value

$$RV_{sector} =  (Sector-Current-Price)/(Sector-200Day-SMA)/(MktCap-weighted-Ave-of Normalized Valuations Across All Sectors) $$

- $RV$ = Relative Value

ACROSS SECTORS:
- We will use the current price of the sector
- The price will be normalize by dividing it by the sector's 200 Day SMA
- We will further divide each sector's current market cap by market cap weighted average of the normalized value of all sectors to establish a cross-sectional relative value measure


In [10]:
# Getting current price of the sector from yahoo finance
sectors = symbols
sector_current_price=[]
for sector in sectors:
    print(sector)
    try:
       sector_current_price.append(yahoo_fin.stock_info.get_live_price(sector))
    except:
        print('Yahoo Finance current price error with: ', sector)

XLB
XLI
XLY
XLP
XLE
XLV
XLF
XLK
XTL
XLU
XLRE


In [11]:
sector_current_price

[80.7699966430664,
 106.08000183105469,
 168.0800018310547,
 70.1500015258789,
 84.58000183105469,
 130.42999267578125,
 35.27000045776367,
 184.77000427246094,
 69.8019027709961,
 62.529998779296875,
 36.18000030517578]

In [12]:
# Getting sector's 200 Day SMA from yahoo finance
sectors = symbols
sector_200DaySMA=[]
for sector in sectors:
    print(sector)
    try:
       sector_200DaySMA.append(yahoo_fin.stock_info.get_data(sector, interval='1d')['close'][-200:].mean())
    except:
        print('Sector_200DaySMA error with: ', sector)

XLB
XLI
XLY
XLP
XLE
XLV
XLF
XLK
XTL
XLU
XLRE


In [13]:
sector_200DaySMA

[80.0763500213623,
 102.76035026550294,
 157.83829925537108,
 72.72189990997315,
 84.7016502380371,
 130.77245048522948,
 33.598299999237064,
 161.539400100708,
 74.30875946044922,
 64.80655012130737,
 36.55230001449585]

In [14]:
#Normalized by dividing it's current price with the sector's 200 Day SMA
normalize=[]
for i in range(11):
    normalize.append(sector_current_price[i]/sector_200DaySMA[i])
normalize

[1.0086623156714694,
 1.032304790290951,
 1.064887309505998,
 0.9646337844957551,
 0.9985638012170891,
 0.9973812694632734,
 1.049755507229966,
 1.143807666471897,
 0.9393495905169578,
 0.9648715857000696,
 0.9898146023869244]

In [15]:
weights=[]
for i in range(11):
    weights.append(market_cap[i]/(np.array(market_cap)).sum())
weights

[0.022987992679551844,
 0.1660982904075347,
 0.13087532919698255,
 0.06294246306298264,
 0.23423202249698702,
 0.068620274070437,
 0.019497388742579117,
 0.060804356559389365,
 0.1600232111770745,
 0.07370441458733205,
 0.0002142570191492211]

In [16]:
weights = np.array(weights)

In [17]:
# Establish a cross-sectional relative value measure
relative_measure=[]
for i in range(10):
    relative_measure.append(normalize[i]/weights.mean())
relative_measure

[11.095285472386163,
 11.35535269320046,
 11.713760404565978,
 10.610971629453307,
 10.98420181338798,
 10.971193964096008,
 11.547310579529626,
 12.581884331190867,
 10.332845495686534,
 10.613587442700766]

In [18]:
relative_scores = dict(zip(symbols, [np.around(r,3) for r in relative_measure]))

In [19]:
relative_scores

{'XLB': 11.095,
 'XLI': 11.355,
 'XLY': 11.714,
 'XLP': 10.611,
 'XLE': 10.984,
 'XLV': 10.971,
 'XLF': 11.547,
 'XLK': 12.582,
 'XTL': 10.333,
 'XLU': 10.614}

In [20]:
# Create a dataframe of computed scores
for asset in relative_scores:
    relative_scores[asset] = [relative_scores[asset]]
scores_df = pd.DataFrame(data = relative_scores, index=['Weighted_score'])
scores_df

,XLB,XLI,XLY,XLP,XLE,XLV,XLF,XLK,XTL,XLU
Weighted_score,11.095,11.355,11.714,10.611,10.984,10.971,11.547,12.582,10.333,10.614
